In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#%cd  /content/drive/"My Drive"/African_Translator/models

In [3]:
#! git clone https://github.com/Tikquuss/meta_XLM /content/meta_XLM

In [5]:
#! cp -R /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs /content

In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [7]:
#%cd tools
#! git clone https://github.com/moses-smt/mosesdecoder
#! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

In [8]:
#%cd -b HOME

In [ ]:
#! mkdir data

In [9]:
#%env csv_path=/content
#%env output_dir=data
#%env data_type=para
#%env languages=Ghomala,Ngiemboon
#! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

In [ ]:
"""
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=data/para/Ghomala-Ngiemboon    
%env MONO_PATH=data/mono   
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=data/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Ghomala-Ngiemboon:-1
"""

In [ ]:
"""
# moins de codes que les autres, sinon "fast: fastBPE/fastBPE.hpp:458: void fastBPE::readCodes(const char* ...).Assertion `codes.find(pair) == codes.end()' failed."
%env nCodes=7000 
%env add_para_data_to_mono_data=False
! ../build_meta_data.sh $sub_task $n_samples $add_para_data_to_mono_data
"""

In [12]:
# save the data for future training
#! mkdir /content/drive/"My Drive"/African_Translator/models/africa/cluster3/data/Ghomala_Ngiemboon
#! rsync -av --progress data/processed /content/drive/"My Drive"/African_Translator/models/africa/cluster3/data/Ghomala_Ngiemboon

In [2]:
%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Ngiemboon/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Ngiemboon/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [2]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [3]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster3/data/Ghomala_Ngiemboon/processed/Ghomala-Ngiemboon.Ghomala.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6339


In [16]:
%env epoch_size=6339

%env lgs=Ghomala-Ngiemboon

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6339
env: lgs=Ghomala-Ngiemboon
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [21]:
%env mlm_steps=Ghomala,Ngiemboon,Ghomala-Ngiemboon
%env batch_size=8
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster3
! python train.py --exp_name mlm_tlm_GhomalaNgiemboon --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Ghomala,Ngiemboon,Ghomala-Ngiemboon
env: batch_size=8
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/20/20 09:05:21 - 0:00:00 - ============ Initialized logger ============
INFO - 05/20/20 09:05:21 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
               

In [18]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_GhomalaNgiemboon  /content/drive/"My Drive"/African_Translator/models/africa/cluster3

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [3]:
%env epoch_size=6360
%env lgs=Ghomala-Ngiemboon

env: epoch_size=6360
env: lgs=Ghomala-Ngiemboon


In [4]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [5]:
%env stopping_criterion=valid_Ghomala-Ngiemboon_mt_bleu,10
%env validation_metrics=valid_Ghomala-Ngiemboon_mt_bleu
#%env reload_model=dumped/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth
%env reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth
%env ae_steps=Ghomala,Ngiemboon
%env bt_steps=Ghomala-Ngiemboon-Ghomala,Ngiemboon-Ghomala-Ngiemboon

env: stopping_criterion=valid_Ghomala-Ngiemboon_mt_bleu,10
env: validation_metrics=valid_Ghomala-Ngiemboon_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster3/mlm_tlm_GhomalaNgiemboon/maml/best-valid_mlm_ppl.pth
env: ae_steps=Ghomala,Ngiemboon
env: bt_steps=Ghomala-Ngiemboon-Ghomala,Ngiemboon-Ghomala-Ngiemboon


In [25]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
#! python train.py --exp_name UnSupMT_BafiaBulu --exp_id $exp_id --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100


In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [6]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=32

%env dump_path=/home/jupyter/models/africa/cluster3

# supervised MT
%env mt_steps=Ghomala-Ngiemboon,Ngiemboon-Ghomala       
! python train.py --exp_name SupMT_GhomalaNgiemboon --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
env: batch_size=32
env: dump_path=/home/jupyter/models/africa/cluster3
env: mt_steps=Ghomala-Ngiemboon,Ngiemboon-Ghomala
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/21/20 05:28:52 - 0:00:00 - ============ Initialized logger ============
INFO - 05/21/20 05:28:52 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Ghomala', 'Ngiemboon']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                       

INFO - 05/21/20 05:28:52 - 0:00:00 - 208849 words (6715 unique) in 6338 sentences. 0 unknown words (0 unique) covering 0.00% of the data.
INFO - 05/21/20 05:28:52 - 0:00:00 - Removed 0 empty sentences.
INFO - 05/21/20 05:28:52 - 0:00:00 - Removed 1 too long sentences.

INFO - 05/21/20 05:28:52 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Ghomala_Ngiemboon/processed/valid.Ghomala.pth ...
INFO - 05/21/20 05:28:52 - 0:00:00 - 26226 words (6715 unique) in 792 sentences. 5 unknown words (5 unique) covering 0.02% of the data.
INFO - 05/21/20 05:28:52 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/21/20 05:28:52 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster3/data/Ghomala_Ngiemboon/processed/test.Ghomala.pth ...
INFO - 05/21/20 05:28:52 - 0:00:00 - 26371 words (6715 unique) in 792 sentences. 7 unknown words (6 unique) covering 0.03% of the data.
INFO - 05/21/20 05:28:52 - 0:00:00 - Removed 0 too long sentences.

INFO - 05/21/20 05:28:52 - 0:

INFO - 05/21/20 05:29:02 - 0:00:10 - Number of parameters (encoder): 82988603
INFO - 05/21/20 05:29:02 - 0:00:10 - Number of parameters (decoder): 108191291
INFO - 05/21/20 05:29:02 - 0:00:10 - Found 0 memories.
INFO - 05/21/20 05:29:02 - 0:00:10 - Found 12 FFN.
INFO - 05/21/20 05:29:02 - 0:00:10 - Found 264 parameters in model.
INFO - 05/21/20 05:29:02 - 0:00:10 - Optimizers: model
WARNING - 05/21/20 05:29:02 - 0:00:10 - Reloading checkpoint from /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/checkpoint.pth ...
WARNING - 05/21/20 05:29:21 - 0:00:29 - Not reloading checkpoint optimizer model.
WARNING - 05/21/20 05:29:21 - 0:00:29 - Reloading 'num_updates' and 'lr' for optimizer model.
WARNING - 05/21/20 05:29:21 - 0:00:29 - Checkpoint reloaded. Resuming at epoch 9 / iteration 405 ...
INFO - 05/21/20 05:29:21 - 0:00:30 - ============ Starting epoch 9 ... ============
INFO - 05/21/20 05:29:21 - 0:00:30 - Creating new training data iterator (ae,Ngiemboon) ...
INFO - 05/2

INFO - 05/21/20 05:36:28 - 0:07:36 -     460 -   49.47 sent/s - 17938.56 words/s - AE-Ghomala:  1.6889 || AE-Ngiemboon:  1.7036 || MT-Ghomala-Ngiemboon:  2.5472 || MT-Ngiemboon-Ghomala:  2.4152 || BT-Ghomala-Ngiemboon-Ghomala:  2.4688 || BT-Ngiemboon-Ghomala-Ngiemboon:  2.7167 -  - model LR: 6.9031e-05
INFO - 05/21/20 05:36:57 - 0:08:06 -     465 -   46.40 sent/s - 19652.63 words/s - AE-Ghomala:  1.5721 || AE-Ngiemboon:  1.6916 || MT-Ghomala-Ngiemboon:  2.6332 || MT-Ngiemboon-Ghomala:  2.2986 || BT-Ghomala-Ngiemboon-Ghomala:  2.5243 || BT-Ngiemboon-Ghomala-Ngiemboon:  2.6880 -  - model LR: 6.9780e-05
INFO - 05/21/20 05:37:28 - 0:08:36 -     470 -   39.36 sent/s - 20629.31 words/s - AE-Ghomala:  1.6219 || AE-Ngiemboon:  1.8833 || MT-Ghomala-Ngiemboon:  2.6284 || MT-Ngiemboon-Ghomala:  2.4857 || BT-Ghomala-Ngiemboon-Ghomala:  2.4897 || BT-Ngiemboon-Ghomala-Ngiemboon:  2.5954 -  - model LR: 7.0530e-05
INFO - 05/21/20 05:37:46 - 0:08:55 - ============ End of epoch 10 ============
INFO - 05

INFO - 05/21/20 05:44:31 - 0:15:39 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 05:44:31 - 0:15:39 - ============ Starting epoch 12 ... ============
INFO - 05/21/20 05:44:48 - 0:15:57 -     510 -   10.66 sent/s -  7657.53 words/s - AE-Ghomala:  1.5095 || AE-Ngiemboon:  1.7049 || MT-Ghomala-Ngiemboon:  2.5522 || MT-Ngiemboon-Ghomala:  2.4722 || BT-Ghomala-Ngiemboon-Ghomala:  2.3764 || BT-Ngiemboon-Ghomala-Ngiemboon:  2.5795 -  - model LR: 7.6524e-05
INFO - 05/21/20 05:45:19 - 0:16:27 -     515 -   46.89 sent/s - 34767.80 words/s - AE-Ghomala:  1.4600 || AE-Ngiemboon:  1.6134 || MT-Ghomala-Ngiemboon:  2.5447 || MT-Ngiemboon-Ghomala:  2.4548 || BT-Ghomala-Ngiemboon-Ghomala:  2.5145 || BT-Ngiemboon-Ghomala-Ngiemboon:  2.6649 -  - model LR: 7.7273e-05
INFO - 05/21/20 05:45:25 - 0:16:33 - Creating new training data iterator (ae,Ghomala) ...
INFO - 05/21/20 05:45:27 - 0:16:36 - Creating new training data iterator (bt,Ghomala) ...
INFO - 05/21/20 05:45:48 - 0:16:56 -     5

...

INFO - 05/21/20 06:43:39 - 1:14:47 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp23.Ngiemboon-Ghomala.valid.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ngiemboon-Ghomala.valid.txt : 7.510000
INFO - 05/21/20 06:43:59 - 1:15:07 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp23.Ghomala-Ngiemboon.valid.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ghomala-Ngiemboon.valid.txt : 6.510000
INFO - 05/21/20 06:44:19 - 1:15:28 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp23.Ngiemboon-Ghomala.test.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ngiemboon-Ghomala.test.txt : 7.820000
INFO - 05/21/20 06:44:39 - 1:15:47 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp23.Ghomala-Ngiemboon.test.txt /home/jupyter/models/africa/cluster3/S

INFO - 05/21/20 06:49:54 - 1:21:03 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 06:49:54 - 1:21:03 - ============ Starting epoch 25 ... ============
INFO - 05/21/20 06:50:00 - 1:21:09 -     950 -   11.50 sent/s - 42273.21 words/s - AE-Ghomala:  0.4161 || AE-Ngiemboon:  0.3618 || MT-Ghomala-Ngiemboon:  0.9563 || MT-Ngiemboon-Ghomala:  0.9519 || BT-Ghomala-Ngiemboon-Ghomala:  1.1334 || BT-Ngiemboon-Ghomala-Ngiemboon:  1.0520 -  - model LR: 8.3771e-05
INFO - 05/21/20 06:50:31 - 1:21:40 -     955 -   46.02 sent/s - 173279.01 words/s - AE-Ghomala:  0.4104 || AE-Ngiemboon:  0.3849 || MT-Ghomala-Ngiemboon:  0.9031 || MT-Ngiemboon-Ghomala:  0.9235 || BT-Ghomala-Ngiemboon-Ghomala:  1.1429 || BT-Ngiemboon-Ghomala-Ngiemboon:  1.0742 -  - model LR: 8.3551e-05
INFO - 05/21/20 06:51:01 - 1:22:10 -     960 -   43.38 sent/s - 180061.95 words/s - AE-Ghomala:  0.3946 || AE-Ngiemboon:  0.3669 || MT-Ghomala-Ngiemboon:  1.0043 || MT-Ngiemboon-Ghomala:  0.9400 || BT-Ghomala-Ngiemboon-Gh

INFO - 05/21/20 06:58:20 - 1:29:28 - ============ End of epoch 26 ============
INFO - 05/21/20 06:58:39 - 1:29:47 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp26.Ngiemboon-Ghomala.valid.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ngiemboon-Ghomala.valid.txt : 7.060000
INFO - 05/21/20 06:59:02 - 1:30:10 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp26.Ghomala-Ngiemboon.valid.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ghomala-Ngiemboon.valid.txt : 5.600000
INFO - 05/21/20 06:59:20 - 1:30:29 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/hyp26.Ngiemboon-Ghomala.test.txt /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hypotheses/ref.Ngiemboon-Ghomala.test.txt : 6.890000
INFO - 05/21/20 06:59:43 - 1:30:51 - BLEU /home/jupyter/models/africa/cluster3/SupMT_GhomalaNgiemboon/maml/hyp

WARNING - 05/21/20 07:04:46 - 1:35:54 - Saving model optimizer ...
INFO - 05/21/20 07:05:04 - 1:36:12 - ============ garbage collector collecting 0 ...
INFO - 05/21/20 07:05:04 - 1:36:12 - ============ Starting epoch 28 ... ============
INFO - 05/21/20 07:05:28 - 1:36:36 -    1055 -   11.48 sent/s - 48615.48 words/s - AE-Ghomala:  0.3295 || AE-Ngiemboon:  0.2777 || MT-Ghomala-Ngiemboon:  0.7738 || MT-Ngiemboon-Ghomala:  0.7912 || BT-Ghomala-Ngiemboon-Ghomala:  0.8823 || BT-Ngiemboon-Ghomala-Ngiemboon:  0.8437 -  - model LR: 7.9493e-05
INFO - 05/21/20 07:05:59 - 1:37:08 -    1060 -   45.62 sent/s - 203072.42 words/s - AE-Ghomala:  0.3047 || AE-Ngiemboon:  0.2724 || MT-Ghomala-Ngiemboon:  0.8110 || MT-Ngiemboon-Ghomala:  0.8094 || BT-Ghomala-Ngiemboon-Ghomala:  0.8627 || BT-Ngiemboon-Ghomala-Ngiemboon:  0.7972 -  - model LR: 7.9305e-05
INFO - 05/21/20 07:06:29 - 1:37:38 -    1065 -   47.60 sent/s - 212900.28 words/s - AE-Ghomala:  0.2880 || AE-Ngiemboon:  0.2835 || MT-Ghomala-Ngiemboon: 

INFO - 05/21/20 07:17:17 - 1:48:25 -    1140 -   50.07 sent/s - 237755.25 words/s - AE-Ghomala:  0.2650 || AE-Ngiemboon:  0.2167 || MT-Ghomala-Ngiemboon:  0.5030 || MT-Ngiemboon-Ghomala:  0.4669 || BT-Ghomala-Ngiemboon-Ghomala:  0.7205 || BT-Ngiemboon-Ghomala-Ngiemboon:  0.5885 -  - model LR: 7.6472e-05
INFO - 05/21/20 07:17:47 - 1:48:56 -    1145 -   48.46 sent/s - 234335.28 words/s - AE-Ghomala:  0.2474 || AE-Ngiemboon:  0.2666 || MT-Ghomala-Ngiemboon:  0.5030 || MT-Ngiemboon-Ghomala:  0.4913 || BT-Ghomala-Ngiemboon-Ghomala:  0.7001 || BT-Ngiemboon-Ghomala-Ngiemboon:  0.6149 -  - model LR: 7.6305e-05
INFO - 05/21/20 07:18:17 - 1:49:25 -    1150 -   49.50 sent/s - 244323.69 words/s - AE-Ghomala:  0.2623 || AE-Ngiemboon:  0.2077 || MT-Ghomala-Ngiemboon:  0.5321 || MT-Ngiemboon-Ghomala:  0.4913 || BT-Ghomala-Ngiemboon-Ghomala:  0.7166 || BT-Ngiemboon-Ghomala-Ngiemboon:  0.5745 -  - model LR: 7.6139e-05
INFO - 05/21/20 07:18:35 - 1:49:43 - ============ End of epoch 30 ============
INFO -

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1